## Introduction

In March 2023, Chancellor Jeremy Hunt [announced](https://assets.publishing.service.gov.uk/media/66221ba8252f0d71cf757d2b/Spring_budget_2023_childcare_expansion_costing_note_information.pdf) an extension of funded childcare hours in England, extending the 30-hours free childcare (the extended childcare entitlement) to children between nine months and four years old (previously, the lower age limit was three years). In this report, we use PolicyEngine to model the fiscal and household impacts of this reform.

## Current policy

Under the previous policy, free childcare hours were available to the following groups:

- All three and four-year-olds were entitled to 15 hours of free childcare per week (the universal entitlement)
- Three and four-year-olds in families where all parents earn the equivalent of 16 hours at the National Minimum Wage were entitled to an additional 15 hours (the extended entitlement)
- Two-year-olds from households claiming specific benefits were eligible for 15 hours of free childcare (the targeted entitlement)

The free entitlement applied for 38 weeks per year (during term time). Parents can choose to spread their hours over more weeks, which reduces the weekly entitlement proportionally. The eligibility threshold for the additional 15 hours includes an income cap, with households where any parent earns over £100,000 not qualifying.

This policy includes several parameters in the PolicyEngine model, including:
- Age thresholds for entitlement (currently set at 1 year for certain provisions)
- Hour allocations for different age brackets
- Income eligibility criteria

## Reform

The reform extended the extended childcare entitlement in three phases:

- **April 2024**: The age eligibility condition is reduced from 3-4 years to 2-4 years. Children aged 2-3 years are entitled to 15 hours of free childcare per week rather than 30 hours.
- **September 2024**: This two-year old age limit is lowered to 9 months. Children aged 9 months to 3 years are entitled to 15 hours of free childcare per week.
- **September 2025**: Children aged 9 months to 4 years are entitled to 30 hours of free childcare per week.

The eligibility criteria for "working parents" remains the same as the current 30-hour offer for three and four-year-olds: parents must earn the equivalent of 16 hours at the National Minimum Wage, and no parent can earn more than £100,000.

Our PolicyEngine model implements this reform by:
1. Reducing the age threshold for childcare entitlement from 1 year to 9 months (0.75 years)
2. Increasing the funded hours for different age groups according to the implementation timeline
3. Maintaining the same income eligibility criteria

## Household impacts

Different household compositions experience varying impacts from the childcare extension. Here are examples based on our model:

**Single parent, one child aged 1, earning £25,000 per year**
- Pre-reform: No entitlement to free childcare
- Post-reform (full implementation): 30 hours free childcare weekly

**Couple, two children aged 10 months and 2 years, combined income of £60,000**
- Pre-reform: No entitlement to free childcare
- Post-reform (full implementation): 30 hours free childcare weekly for both children

The reform affects households differently based on family composition, income levels, and childcare usage patterns. Families with incomes over £100,000 are not eligible for the extended entitlement.

## Economic impacts

Our analysis shows that the childcare hours extension will reduce government revenue by the following amounts:

| Revenue impact (£bn) | 2024/25 | 2025/26 | 2026/27 | 2027/28 | 2028/29 | 2029/30 |
|:---------------------|-----:|-----:|-----:|-----:|-----:|-----:|
| PolicyEngine | -1.3 | -3.9 | -4.3 | -4.4 | -4.6 | -4.7 |
| HM Treasury | -1.7 | -3.3 | -4.1 | -4.1 | - | - |

## Conclusion

We estimate that the UK's extended childcare entitlement will reduce government revenue by £4.7 billion in 2029/30, and will increase the net income of households with children who meet the eligibility criteria. The greatest income changes accrue to families with multiple young children who use the full entitlement.

Use PolicyEngine to view the full results or calculate the effect on your household.



In [91]:
baseline = {
    "gov.dfe.extended_childcare_entitlement.hours[1].threshold": {
        "2024": 1,
        "2025": 1,
        "2026": 1,
        "2027": 1,
        "2028": 1,
        "2029": 1,
    },
    "gov.dfe.extended_childcare_entitlement.hours[1].amount": {
        "2024": 0,
        "2025": 0,
        "2026": 0,
        "2027": 0,
        "2028": 0,
        "2029": 0,
    },
    "gov.dfe.extended_childcare_entitlement.hours[2].amount": {
        "2024": 0,
        "2025": 0,
        "2026": 0,
        "2027": 0,
        "2028": 0,
        "2029": 0,
    },
}
reform = {
    "gov.dfe.extended_childcare_entitlement.hours[1].threshold": {
        "2024": 0.75,
        "2025": 0.75,
        "2026": 0.75,
        "2027": 0.75,
        "2028": 0.75,
        "2029": 0.75,
    },
    "gov.dfe.extended_childcare_entitlement.hours[0].amount": {
        "2024": (0 + 15) / 2,
        "2025": (15 + 30) / 2,
        "2026": 30,
        "2027": 30,
        "2028": 30,
        "2029": 30,
    },
    "gov.dfe.extended_childcare_entitlement.hours[2].amount": {
        "2024": 15,
        "2025": (15 + 30) / 2,
        "2026": 300,
        "2027": 30,
        "2028": 30,
        "2029": 30,
    },
}

from policyengine import Simulation

sim = Simulation(
    country="uk",
    scope="macro",
    baseline=baseline,
    reform=reform,
)
baseline = sim.baseline_simulation
reformed = sim.reform_simulation

costs_per_year = []
for year in range(2024, 2030):
    baseline_spending = baseline.calculate("gov_spending", year).sum()/1e9
    reform_spending = reformed.calculate("gov_spending", year).sum()/1e9
    change = reform_spending - baseline_spending
    costs_per_year.append(change)

from policyengine.utils.charts import *

Using Hugging Face for download.


In [93]:
comparison = sim.calculate_economy_comparison()

Using Hugging Face for download.
Using Hugging Face for download.


In [104]:
fig = px.bar(
    x=comparison.decile.average.keys(),
    y=comparison.decile.average.values(),
    color_discrete_sequence=[BLUE]
)

format_fig(fig).update_layout(
    title="Impact of the 2023 childcare expansion by income decile"
)

In [48]:
import pandas as pd

df = pd.DataFrame({
    "Year": [str(year) for year in range(2024, 2030)],
    "Revenue impact (£bn)": costs_per_year,
})
df["Revenue impact (£bn)"] = df["Revenue impact (£bn)"].round(1)
df["Revenue impact (£bn)"] *= -1

print(df.set_index("Year").T.to_markdown(index=True))

|                      |   2024 |   2025 |   2026 |   2027 |   2028 |   2029 |
|:---------------------|-------:|-------:|-------:|-------:|-------:|-------:|
| Revenue impact (£bn) |   -1.3 |   -3.9 |   -4.3 |   -4.4 |   -4.6 |   -4.7 |


In [64]:
baseline_df = baseline.calculate_dataframe(["age", "household_net_income", "household_id", "extended_childcare_entitlement"], 2029)
reformed_df = reformed.calculate_dataframe(["age", "household_net_income", "household_id", "extended_childcare_entitlement"], 2029)
baseline_df = baseline_df.rename(columns={col: f"baseline_{col}" for col in baseline_df.columns})
reformed_df = reformed_df.rename(columns={col: f"reformed_{col}" for col in reformed_df.columns})

df = pd.concat([baseline_df, reformed_df], axis=1)
df["change"] = df["reformed_household_net_income"] - df["baseline_household_net_income"]

In [68]:
df[df.baseline_household_id.isin(df[df.change.between(1, 2000)].sample(1).baseline_household_id.values)]

,baseline_age,baseline_household_net_income,baseline_household_id,baseline_extended_childcare_entitlement,reformed_age,reformed_household_net_income,reformed_household_id,reformed_extended_childcare_entitlement,change
20701,29.0,55654.0,10974300,0.0,29.0,57326.0,10974300,1672.24939,1672.0
20702,33.0,55654.0,10974300,0.0,33.0,57326.0,10974300,1672.24939,1672.0
20703,0.0,55654.0,10974300,0.0,0.0,57326.0,10974300,1672.24939,1672.0


In [71]:
df[df.baseline_household_id.isin(df[df.change.between(2001, 8000)].sample(1).baseline_household_id.values)]

,baseline_age,baseline_household_net_income,baseline_household_id,baseline_extended_childcare_entitlement,reformed_age,reformed_household_net_income,reformed_household_id,reformed_extended_childcare_entitlement,change
153404,37.0,114136.0,34674100,0.0,37.0,117263.0,34674100,3127.197266,3127.0
153405,38.0,114136.0,34674100,0.0,38.0,117263.0,34674100,3127.197266,3127.0
153406,1.0,114136.0,34674100,0.0,1.0,117263.0,34674100,3127.197266,3127.0


In [89]:
baseline = {
    "gov.dfe.extended_childcare_entitlement.hours[1].threshold": {
        "2024": 1,
        "2025": 1,
        "2026": 1,
        "2027": 1,
        "2028": 1,
        "2029": 1,
    },
    "gov.dfe.extended_childcare_entitlement.hours[1].amount": {
        "2024": 0,
        "2025": 0,
        "2026": 0,
        "2027": 0,
        "2028": 0,
        "2029": 0,
    },
    "gov.dfe.extended_childcare_entitlement.hours[2].amount": {
        "2024": 0,
        "2025": 0,
        "2026": 0,
        "2027": 0,
        "2028": 0,
        "2029": 0,
    },
}
reform = {
    "gov.dfe.extended_childcare_entitlement.hours[1].threshold": {
        "2024": 0.75,
        "2025": 0.75,
        "2026": 0.75,
        "2027": 0.75,
        "2028": 0.75,
        "2029": 0.75,
    },
    "gov.dfe.extended_childcare_entitlement.hours[0].amount": {
        "2024": (0 + 15) / 2,
        "2025": (15 + 30) / 2,
        "2026": 30,
        "2027": 30,
        "2028": 30,
        "2029": 30,
    },
    "gov.dfe.extended_childcare_entitlement.hours[2].amount": {
        "2024": 15,
        "2025": (15 + 30) / 2,
        "2026": 300,
        "2027": 30,
        "2028": 30,
        "2029": 30,
    },
}

def get_array(
    year: int,
    child_age: int,
    is_reform: bool,
):
    household = Simulation(
        country="uk",
        scope="household",
        baseline=reform if is_reform else baseline,
        data={
            "households": {
                "household": {
                    "members": ["parent", "child"]
                }
            },
            "people": {
                "parent": {
                    "age": 30,
                },
                "child": {
                    "age": child_age,
                }
            },
            "axes": [[{
                "name": "employment_income",
                "min": 0,
                "max": 150_000,
                "count": 151,
            }]]
        }
    )

    return household.baseline_simulation.calculate("household_net_income", year)
    

household = Simulation(
    country="uk",
    scope="household",
    baseline=baseline,
    reform=reform,
    data={
        "households": {
            "household": {
                "members": ["parent", "child"]
            }
        },
        "people": {
            "parent": {
                "age": 30,
            },
            "child": {
                "age": 2,
            }
        },
        "axes": [[{
            "name": "employment_income",
            "min": 0,
            "max": 150_000,
            "count": 151,
        }]]
    }
)

market_income = household.baseline_simulation.calculate("household_market_income", 2029)

baseline_values = []
child_age_values = []
year_values = []
hnet_values = []
market_values = []

for is_baseline in [True]:
    for child_age in [0.75, 1, 2]:
        for year in range(2024, 2030):
            array_r = get_array(year, child_age, True)
            array_b = get_array(year, child_age, False)
            for i in range(len(array_r)):
                baseline_values.append(is_baseline)
                child_age_values.append(child_age)
                year_values.append(year)
                hnet_values.append(array_r[i] - array_b[i])
                market_values.append(market_income[i])

df = pd.DataFrame({
    "Baseline": baseline_values,
    "Child age": child_age_values,
    "Year": year_values,
    "Household net income": hnet_values,
    "Market income": market_values,
})

In [1]:
px.line(
    df[df.Year == 2029],
    x="Market income",
    y="Household net income",
    color="Child age",
)

NameError: name 'px' is not defined